In [1]:
import re
import string
from natasha import MorphVocab
from loguru import logger
import numpy as np
import pandas as pd
from razdel import tokenize

In [2]:
vectorized_news_df = pd.read_csv('reduced_news.csv', delimiter =',')



In [3]:
vectorized_news_df = vectorized_news_df.head()
vectorized_news_df

,Unnamed: 0,url,title,text,topic,tags,date
0,0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15
1,1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15
2,2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Культура,2019-12-15
3,4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15
4,5,https://lenta.ru/news/2019/12/15/apple/,Свадьба трансгендеров в России попала на видео,В ЗАГСе Казани официально вступила в брак пара...,Россия,Общество,2019-12-15


In [8]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
from navec import Navec

# Создание объектов для сегментации и морфологического анализа
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

# Функция для лемматизации текста
def lemmatize_text(text):
    # Морфологический анализ сегментов
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    lemmas = []
    vectors = []
    for token in doc.tokens:
        if token.pos != 'PUNCT':
            # Получение леммы для каждого токена
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
            if token.lemma in emb:
                vector = emb[token.lemma]
                vectors.append(vector)
            else:
                vectors.append(np.zeros(300).astype(float))
    vectors = np.asarray(vectors)
    vectors.mean(axis=0)
    return vectors
# Пример использования функции для лемматизации текста
texts =  [ "Украинская ЛГБТ-активистка обвинила ню-фотографа...", "Маму ебал", "А как она сдох" ]
vectorized_news_df['vectors'] = [lemmatize_text(text) for text in vectorized_news_df['text']] 
vectorized_news_df

,Unnamed: 0,url,title,text,topic,tags,date,vectors
0,0,https://lenta.ru/news/2019/12/15/prsm/,Россиянам дали советы по выбору чая,Россиянам дали советы при выборе чая. Рекоменд...,Россия,Общество,2019-12-15,"[[0.33644357323646545, -0.18003053963184357, 0..."
1,1,https://lenta.ru/news/2019/12/15/fb/,В Госдуме назвали японское заявление о Курилах...,Спикер Госдумы Вячеслав Володин назвал угрозой...,Россия,Политика,2019-12-15,"[[0.1074577048420906, 0.10019926726818085, -0...."
2,2,https://lenta.ru/news/2019/12/15/kino/,Украинская ЛГБТ-активистка обвинила ню-фотогра...,Украинская ЛГБТ-активистка Виктория Гуйвик обв...,Культура,Культура,2019-12-15,"[[-0.6066527962684631, -0.10343946516513824, 0..."
3,4,https://lenta.ru/news/2019/12/15/anons/,Беглого президента Боливии решили арестовать,Исполняющая обязанности президента Боливии Жан...,Мир,Политика,2019-12-15,"[[0.33644357323646545, -0.18003053963184357, 0..."
4,5,https://lenta.ru/news/2019/12/15/apple/,Свадьба трансгендеров в России попала на видео,В ЗАГСе Казани официально вступила в брак пара...,Россия,Общество,2019-12-15,"[[-0.19732695817947388, 0.09466413408517838, -..."
